<a href="https://colab.research.google.com/github/rlatpwls7337/git_test/blob/main/ESG_VGG%EA%B5%AC%ED%98%84(%EC%88%98%EC%97%85%EC%9A%A9)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG CNN(Convolution Neural Network) 구현하기 with keras

In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.9.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.9.2


In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 1.7 MB 59.9 MB/s 
     |████████████████████████████████| 5.9 MB 53.9 MB/s 
     |████████████████████████████████| 438 kB 69.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

# Image Classification Using VGG - Cat or Dog Kaggle Competition

![catanddog](https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg)

* 출처 : kaggle competitions(dogs vs cats)

##### * [Cat of Dog Kaggel competition](https://www.kaggle.com/c/dogs-vs-cats/overview/description)를 이용한 이미지 분류 프로젝트.
##### * VGG16 모델을 이용하여 학습
##### * VGG-16 paper : proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper “[Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)”.
##### * patrick013 code

In [ ]:
import pandas as pd
import IPython.display as display
import tensorflow as tf
from tensorflow.keras import layers, activations
import numpy as np
import os, random
import matplotlib.pyplot as plt
print(tf.__version__)

2.10.0


## Prepare Data

Dataset
* training set : 25,000 이미지
* testing set : 12,500 이미지
* class : dogs, cats
Since uploading the dataset to google colab is time-consuming.


In [ ]:
! git clone https://github.com/patrick013/Image-Classification-CNN-and-VGG.git

Cloning into 'Image-Classification-CNN-and-VGG'...
remote: Enumerating objects: 37510, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 37510 (delta 43), reused 36 (delta 7), pack-reused 37414
Receiving objects: 100% (37510/37510), 812.51 MiB | 36.43 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Checking out files: 100% (50016/50016), done.


In [ ]:
IMAGE_HEIGHT=128
IMAGE_WIDTH=128
BATCH_SIZE=64
def get_pathframe(path):
  '''
  이미지 경로, 라벨, 파일이름 
  pandas dataframe에 모두 저장
  '''
  filenames = os.listdir(path)
  categories = []
  paths=[]
  for filename in filenames:
    paths.append(path+filename)
    category = filename.split('.')[0]
    if category == 'dog':
      categories.append(1)
    else:
      categories.append(0)

  df= pd.DataFrame({
      'filename': filenames,
      'category': categories,
      'paths':paths
  })
  return df

In [ ]:
df=get_pathframe("Image-Classification-CNN-and-VGG/dataset/dataset/")
df.tail(5)

,filename,category,paths
24995,dog.12375.jpg,1,Image-Classification-CNN-and-VGG/dataset/datas...
24996,dog.7860.jpg,1,Image-Classification-CNN-and-VGG/dataset/datas...
24997,cat.11772.jpg,0,Image-Classification-CNN-and-VGG/dataset/datas...
24998,cat.6889.jpg,0,Image-Classification-CNN-and-VGG/dataset/datas...
24999,dog.630.jpg,1,Image-Classification-CNN-and-VGG/dataset/datas...


In [ ]:
df['paths'][0]

'Image-Classification-CNN-and-VGG/dataset/dataset/cat.6490.jpg'

In [ ]:
def load_and_preprocess_image(path):
    '''
    불러온 이미지를 resize하는 함수 
    '''
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image,[IMAGE_WIDTH,IMAGE_HEIGHT]) 
    image = image/255.0
    return image

def convert_to_tensor(df):
    '''
    데이터와 라벨을 tensor로 변환
    '''
    path_ds = tf.data.Dataset.from_tensor_slices(df['paths'])
    image_ds = path_ds.map(load_and_preprocess_image)
    # onehot_label = tf.one_hot(tf.cast(df['category'],tf.int64,2)) # if using softmax 
    onehot_label = tf.cast(df['category'],tf.int64)
    label_ds = tf.data.Dataset.from_tensor_slices(onehot_label)

    return image_ds,label_ds

In [ ]:
X,Y = convert_to_tensor(df)
print(X)
print('\n')
print(Y)

<MapDataset element_spec=TensorSpec(shape=(128, 128, 3), dtype=tf.float32, name=None)>


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [ ]:
dataset = tf.data.Dataset.zip((X,Y)).shuffle(buffer_size=20)
dataset_train = dataset.take(22500)
dataset_test = dataset.skip(22500)

dataset_train = dataset_train.batch(BATCH_SIZE,drop_remainder=True)
dataset_test = dataset_test.batch(BATCH_SIZE,drop_remainder=True)
dataset_train

<BatchDataset element_spec=(TensorSpec(shape=(64, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(64,), dtype=tf.int64, name=None))>

### Check Images

##### tesorflow conv2D 형태 예시
```
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
)
```

## Convolutional Neural Networks - VGGNet
### Build Network


### Training

### Evaluation

## VGG16 API 활용


### Training

### Evaluation
